In [1]:
# Import dependencies
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
#from tensorflow.keras.callbacks import ModelCheckpoint
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from config import pw_postgres

ModuleNotFoundError: No module named 'config'

In [ ]:
# Import and read data

DB_PASS = pw_postgres
aws_database = 'shrbfk-final-project.cuitpsvagrne.us-east-2.rds.amazonaws.com:5432'
db_string = "postgresql+psycopg2://postgres:" + DB_PASS + "@" + aws_database + "/vet_data"
try:
    db_engine = create_engine(db_string)
    
except Exception as e:
    print(f"\nFailed to create database connection to {aws_database}.\n", e)
    exit()
    
combined_df = pd.read_sql_table('people_vet_join', db_engine)

In [ ]:
combined_df.head()

In [ ]:
df_labels = combined_df[['State', 'County']]
combined_df = combined_df.drop(columns=['FIPS', 'State', 'County'])
#combined_df = combined_df.drop

In [ ]:
combined_df = combined_df.drop([0,68,102,118,194,253,318,331,333,401,561,567,612,715,808,908,1014,
               1135,1200,1217,1242,1257,1341,1429,1512,1628,1685,1779,1797,1808,1830,
               1864,1927,2028,2082,2171,2249,2286,2354,2360,2407,2474,2570,2825,
               2855,2870,3005,3045,3101,3174])

In [ ]:
# Drop non-helpful columns(if there are any)
#combined_df.drop(columns=['columnname1', 'columnname2'], axis=1, inplace=True)
combined_df.head()

In [ ]:
combined_df.dropna(inplace=True)

In [ ]:
# Split into features and target arrays
y = combined_df['TotalVets'].values
X = combined_df.drop(['TotalVets'],1).values

In [ ]:
# Split preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split( X, y, random_state=42)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
# Fit the model
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
lin_reg_y_pred = lin_reg.predict(X_test_scaled)
mse = mean_squared_error(y_test, lin_reg_y_pred)
rmse = np.sqrt(mse)
print(rmse)
#print(mse)

In [ ]:
r2_score(y_test, lin_reg_y_pred)

In [ ]:
plt.hist(y, 10, range=[0,15000])
plt.show()

In [ ]:
#STOP HERE 

In [ ]:
#y_test

In [ ]:
#y

In [ ]:
# Fitting Random Forest Regression to the dataset
#regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
#regressor.fit(X.reshape(-1,1), y.reshape(-1, 1))

In [ ]:
#ridge = Ridge()
#ridge.fit(X_train, y_train)
#y_pred_ridge = ridge.predict(X_test)
#mse = mean_squared_error(y_test, y_pred_ridge)
#rmse = np.sqrt(mse)
#print(rmse)

In [ ]:
#lasso = Lasso()
#lasso.fit(X_train, y_train)
#y_pred_lasso = lasso.predict(X_test)
#mse = mean_squared_error(y_test, y_pred_lasso)
#print(mse)